# Load and plot a Simtel image using PyWI-CTA

## Import required modules

This notebook requires PyWI-CTA for the I/O.

In [ ]:
%matplotlib inline

import pywicta
from pywicta.io import geometry_converter
from pywicta.io.images import image_generator
from pywicta.io.images import plot_ctapipe_image
from pywicta.io.images import get_mars_like_default_integrator_config
from pywicta.denoising.rejection_criteria import CTAMarsCriteria

print(pywicta.get_version())

In [ ]:
pywicta.__file__

## Setup

The next cells define the list of images to use in this notebook.

Images can be fetched from Fits files or from Simtel files.
Fits files are much lighter and much faster to process than Simtel files but they are specific to PyWI-CTA and thus you first have to generate them from Simtel files using [the following script](). Also, contrary to Simtel files, a Fits files contains only one "image" (i.e. an unique event viewed from one unique telescope).

In [ ]:
#CAM_ID = "ASTRICam"
#CAM_ID = "CHEC"
#CAM_ID = "DigiCam"
#CAM_ID = "FlashCam"
#CAM_ID = "NectarCam"
CAM_ID = "LSTCam"

IMG_PATH_LIST = ["~/proton_20deg_0deg_run50001___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz"]
NUM_IMAGES = 1
TEL_FILTER_LIST = []
EVENT_FILTER_LIST = []

## Define preselection cuts

image_generator will only returns images that pass the following rejection_criteria (rejection criteria is a function that takes an image in argument and return True if the image pass the test of False otherwise).

In [ ]:
#rejection_criteria = lambda ref_image: not 200 < np.nansum(ref_image) < 250
rejection_criteria = CTAMarsCriteria(cam_id=CAM_ID)
#rejection_criteria = None

## Define the calibration / integration setup

In [ ]:
integrator_config_dict = get_mars_like_default_integrator_config(CAM_ID)

## Get image

In [ ]:
image_gen = image_generator(IMG_PATH_LIST,
                            max_num_images=NUM_IMAGES,
                            cam_filter_list=[CAM_ID],
                            tel_filter_list=TEL_FILTER_LIST,
                            ev_filter_list=EVENT_FILTER_LIST,
                            ctapipe_format=False,
                            mc_rejection_criteria=rejection_criteria,
                            **integrator_config_dict)

In [ ]:
img = next(image_gen)

In [ ]:
img

In [ ]:
img.meta

## Plot using pywicta

In [ ]:
geom1d = geometry_converter.get_geom1d(CAM_ID)

img_input_1d = geometry_converter.image_2d_to_1d(img.input_image, CAM_ID)
img_ref_1d = geometry_converter.image_2d_to_1d(img.reference_image, CAM_ID)

In [ ]:
disp = plot_ctapipe_image(img_input_1d,
                          geom1d,
                          title='Noised image',
                          norm='lin',
                          plot_axis=False)

In [ ]:
disp = plot_ctapipe_image(img_ref_1d,
                          geom1d,
                          title='Clean image',
                          norm='lin',
                          plot_axis=False)

In [ ]:
pywicta.io.images.print_hillas_parameters(img_ref_1d, CAM_ID)